# Create Submission From Model

Run inference on the test images to create results ready for submission. This will create a folder called `submission` in the current directory.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch, torchvision
torch.__version__

'1.3.1'

In [3]:
from detectron2.evaluation.xview_evaluation import *

In [4]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

import os
import numpy as np
import json
import shutil
import cv2
import random
import glob

import matplotlib.pyplot as plt
%matplotlib inline

/home/ethanweber/anaconda3/lib/python3.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [5]:
# Set some parameters.
LOCALIZATION_SUBMISSION_FOLDER = "SUBMISSION_LOCALIZATION"
LOCALIZATION_MODEL_CONFIG = "./configs/xview/localization-10.yaml"
DAMAGE_SUBMISSION_FOLDER = "SUBMISSION_DAMAGE"
DAMAGE_MODEL_CONFIG = "./configs/xview/mask_rcnn_R_50_FPN_1x-damage-00.yaml"

In [6]:
# Removes and creates folders (clears them).
shutil.rmtree(LOCALIZATION_SUBMISSION_FOLDER, ignore_errors=True)
shutil.rmtree(DAMAGE_SUBMISSION_FOLDER, ignore_errors=True)
os.mkdir(LOCALIZATION_SUBMISSION_FOLDER)
os.mkdir(DAMAGE_SUBMISSION_FOLDER)

In [7]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

# Load localization model.
cfg_localization = get_cfg()
cfg_localization.merge_from_file(LOCALIZATION_MODEL_CONFIG)
# Load localization checkpoint.
cfg_localization.MODEL.WEIGHTS = os.path.join(cfg_localization.OUTPUT_DIR, "model_final.pth")
# cfg_localization.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 # set the testing threshold for this model
predictor_localization = DefaultPredictor(cfg_localization)

# Load damage model.
cfg_damage = get_cfg()
cfg_damage.merge_from_file(DAMAGE_MODEL_CONFIG)
# Load damage checkpoint.
cfg_damage.MODEL.WEIGHTS = os.path.join(cfg_damage.OUTPUT_DIR, "model_0089999.pth")
# cfg_damage.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 # set the testing threshold for this model
predictor_damage = DefaultPredictor(cfg_damage)

WARNING [12/22 17:29:48 d2.config.compat]: Config './configs/xview/localization-10.yaml' has no VERSION. Assuming it to be compatible with latest v2.
WARNING [12/22 17:29:50 d2.config.compat]: Config './configs/xview/mask_rcnn_R_50_FPN_1x-damage-00.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [8]:
image_filenames = sorted(glob.glob("data/test/images/*"))
for filename in image_filenames:
    # Load the image.
    image = cv2.imread(filename)
    
    # Choose the model type based on pre or post localization
    if filename.find("pre") >= 0:
        # LOCALIZATION model.
        outputs = predictor_localization(image)
    
        # get to the right format
        instances = outputs["instances"].to('cpu')
        # use RLE to encode the masks, because they are too large and takes memory
        # since this evaluator stores outputs of the entire dataset
        # Our model may predict bool array, but cocoapi expects uint8
        rles = [
            mask_util.encode(np.array(mask[:, :, None], order="F", dtype="uint8"))[0]
            for mask in instances.pred_masks
        ]
        for rle in rles:
            # "counts" is an array encoded by mask_util as a byte-stream. Python3's
            # json writer which always produces strings cannot serialize a bytestream
            # unless you decode it. Thankfully, utf-8 works out (which is also what
            # the pycocotools/_mask.pyx does).
            rle["counts"] = rle["counts"].decode("utf-8")
        instances.pred_masks_rle = rles
        instances.remove("pred_masks")
        prediction = {}
        prediction["instances"] = instances_to_json(instances)
        height = 1024
        width = 1024
        
        pred_image = get_xview_localization_pred_image(height, width, prediction)
        
        name = os.path.basename(filename)
        new_filename = name.replace("pre", "localization")[:-4] + "_prediction.png"
        submission_filename = os.path.join(LOCALIZATION_SUBMISSION_FOLDER, new_filename)
        
        print(submission_filename)
        cv2.imwrite(submission_filename, pred_image)
        
    elif filename.find("post") >= 0:
        # DAMAGE model.
        outputs = predictor_damage(image)
    
        # get to the right format
        instances = outputs["instances"].to('cpu')
        # use RLE to encode the masks, because they are too large and takes memory
        # since this evaluator stores outputs of the entire dataset
        # Our model may predict bool array, but cocoapi expects uint8
        rles = [
            mask_util.encode(np.array(mask[:, :, None], order="F", dtype="uint8"))[0]
            for mask in instances.pred_masks
        ]
        for rle in rles:
            # "counts" is an array encoded by mask_util as a byte-stream. Python3's
            # json writer which always produces strings cannot serialize a bytestream
            # unless you decode it. Thankfully, utf-8 works out (which is also what
            # the pycocotools/_mask.pyx does).
            rle["counts"] = rle["counts"].decode("utf-8")
        instances.pred_masks_rle = rles
        instances.remove("pred_masks")
        prediction = {}
        prediction["instances"] = instances_to_json(instances)
        height = 1024
        width = 1024
        
        pred_image = get_xview_damage_pred_image(height, width, prediction)
        
        name = os.path.basename(filename)
        new_filename = name.replace("post", "damage")[:-4] + "_prediction.png"
        submission_filename = os.path.join(DAMAGE_SUBMISSION_FOLDER, new_filename)
        
        print(submission_filename)
        cv2.imwrite(submission_filename, pred_image)



SUBMISSION_DAMAGE/test_damage_00000_prediction.png
SUBMISSION_DAMAGE/test_damage_00001_prediction.png
SUBMISSION_DAMAGE/test_damage_00002_prediction.png
SUBMISSION_DAMAGE/test_damage_00003_prediction.png
SUBMISSION_DAMAGE/test_damage_00004_prediction.png
SUBMISSION_DAMAGE/test_damage_00005_prediction.png
SUBMISSION_DAMAGE/test_damage_00006_prediction.png
SUBMISSION_DAMAGE/test_damage_00007_prediction.png
SUBMISSION_DAMAGE/test_damage_00008_prediction.png
SUBMISSION_DAMAGE/test_damage_00009_prediction.png
SUBMISSION_DAMAGE/test_damage_00010_prediction.png
SUBMISSION_DAMAGE/test_damage_00011_prediction.png
SUBMISSION_DAMAGE/test_damage_00012_prediction.png
SUBMISSION_DAMAGE/test_damage_00013_prediction.png
SUBMISSION_DAMAGE/test_damage_00014_prediction.png
SUBMISSION_DAMAGE/test_damage_00015_prediction.png
SUBMISSION_DAMAGE/test_damage_00016_prediction.png
SUBMISSION_DAMAGE/test_damage_00017_prediction.png
SUBMISSION_DAMAGE/test_damage_00018_prediction.png
SUBMISSION_DAMAGE/test_damage_0

SUBMISSION_DAMAGE/test_damage_00161_prediction.png
SUBMISSION_DAMAGE/test_damage_00162_prediction.png
SUBMISSION_DAMAGE/test_damage_00163_prediction.png
SUBMISSION_DAMAGE/test_damage_00164_prediction.png
SUBMISSION_DAMAGE/test_damage_00165_prediction.png
SUBMISSION_DAMAGE/test_damage_00166_prediction.png
SUBMISSION_DAMAGE/test_damage_00167_prediction.png
SUBMISSION_DAMAGE/test_damage_00168_prediction.png
SUBMISSION_DAMAGE/test_damage_00169_prediction.png
SUBMISSION_DAMAGE/test_damage_00170_prediction.png
SUBMISSION_DAMAGE/test_damage_00171_prediction.png
SUBMISSION_DAMAGE/test_damage_00172_prediction.png
SUBMISSION_DAMAGE/test_damage_00173_prediction.png
SUBMISSION_DAMAGE/test_damage_00174_prediction.png
SUBMISSION_DAMAGE/test_damage_00175_prediction.png
SUBMISSION_DAMAGE/test_damage_00176_prediction.png
SUBMISSION_DAMAGE/test_damage_00177_prediction.png
SUBMISSION_DAMAGE/test_damage_00178_prediction.png
SUBMISSION_DAMAGE/test_damage_00179_prediction.png
SUBMISSION_DAMAGE/test_damage_0

SUBMISSION_DAMAGE/test_damage_00323_prediction.png
SUBMISSION_DAMAGE/test_damage_00324_prediction.png
SUBMISSION_DAMAGE/test_damage_00325_prediction.png
SUBMISSION_DAMAGE/test_damage_00326_prediction.png
SUBMISSION_DAMAGE/test_damage_00327_prediction.png
SUBMISSION_DAMAGE/test_damage_00328_prediction.png
SUBMISSION_DAMAGE/test_damage_00329_prediction.png
SUBMISSION_DAMAGE/test_damage_00330_prediction.png
SUBMISSION_DAMAGE/test_damage_00331_prediction.png
SUBMISSION_DAMAGE/test_damage_00332_prediction.png
SUBMISSION_DAMAGE/test_damage_00333_prediction.png
SUBMISSION_DAMAGE/test_damage_00334_prediction.png
SUBMISSION_DAMAGE/test_damage_00335_prediction.png
SUBMISSION_DAMAGE/test_damage_00336_prediction.png
SUBMISSION_DAMAGE/test_damage_00337_prediction.png
SUBMISSION_DAMAGE/test_damage_00338_prediction.png
SUBMISSION_DAMAGE/test_damage_00339_prediction.png
SUBMISSION_DAMAGE/test_damage_00340_prediction.png
SUBMISSION_DAMAGE/test_damage_00341_prediction.png
SUBMISSION_DAMAGE/test_damage_0

SUBMISSION_DAMAGE/test_damage_00484_prediction.png
SUBMISSION_DAMAGE/test_damage_00485_prediction.png
SUBMISSION_DAMAGE/test_damage_00486_prediction.png
SUBMISSION_DAMAGE/test_damage_00487_prediction.png
SUBMISSION_DAMAGE/test_damage_00488_prediction.png
SUBMISSION_DAMAGE/test_damage_00489_prediction.png
SUBMISSION_DAMAGE/test_damage_00490_prediction.png
SUBMISSION_DAMAGE/test_damage_00491_prediction.png
SUBMISSION_DAMAGE/test_damage_00492_prediction.png
SUBMISSION_DAMAGE/test_damage_00493_prediction.png
SUBMISSION_DAMAGE/test_damage_00494_prediction.png
SUBMISSION_DAMAGE/test_damage_00495_prediction.png
SUBMISSION_DAMAGE/test_damage_00496_prediction.png
SUBMISSION_DAMAGE/test_damage_00497_prediction.png
SUBMISSION_DAMAGE/test_damage_00498_prediction.png
SUBMISSION_DAMAGE/test_damage_00499_prediction.png
SUBMISSION_DAMAGE/test_damage_00500_prediction.png
SUBMISSION_DAMAGE/test_damage_00501_prediction.png
SUBMISSION_DAMAGE/test_damage_00502_prediction.png
SUBMISSION_DAMAGE/test_damage_0

SUBMISSION_DAMAGE/test_damage_00645_prediction.png
SUBMISSION_DAMAGE/test_damage_00646_prediction.png
SUBMISSION_DAMAGE/test_damage_00647_prediction.png
SUBMISSION_DAMAGE/test_damage_00648_prediction.png
SUBMISSION_DAMAGE/test_damage_00649_prediction.png
SUBMISSION_DAMAGE/test_damage_00650_prediction.png
SUBMISSION_DAMAGE/test_damage_00651_prediction.png
SUBMISSION_DAMAGE/test_damage_00652_prediction.png
SUBMISSION_DAMAGE/test_damage_00653_prediction.png
SUBMISSION_DAMAGE/test_damage_00654_prediction.png
SUBMISSION_DAMAGE/test_damage_00655_prediction.png
SUBMISSION_DAMAGE/test_damage_00656_prediction.png
SUBMISSION_DAMAGE/test_damage_00657_prediction.png
SUBMISSION_DAMAGE/test_damage_00658_prediction.png
SUBMISSION_DAMAGE/test_damage_00659_prediction.png
SUBMISSION_DAMAGE/test_damage_00660_prediction.png
SUBMISSION_DAMAGE/test_damage_00661_prediction.png
SUBMISSION_DAMAGE/test_damage_00662_prediction.png
SUBMISSION_DAMAGE/test_damage_00663_prediction.png
SUBMISSION_DAMAGE/test_damage_0

...

SUBMISSION_LOCALIZATION/test_localization_00291_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00292_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00293_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00294_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00295_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00296_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00297_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00298_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00299_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00300_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00301_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00302_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00303_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00304_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00305_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00306_predict

SUBMISSION_LOCALIZATION/test_localization_00422_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00423_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00424_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00425_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00426_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00427_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00428_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00429_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00430_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00431_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00432_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00433_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00434_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00435_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00436_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00437_predict

SUBMISSION_LOCALIZATION/test_localization_00553_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00554_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00555_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00556_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00557_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00558_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00559_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00560_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00561_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00562_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00563_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00564_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00565_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00566_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00567_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00568_predict

SUBMISSION_LOCALIZATION/test_localization_00685_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00686_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00687_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00688_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00689_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00690_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00691_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00692_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00693_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00694_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00695_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00696_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00697_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00698_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00699_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00700_predict

SUBMISSION_LOCALIZATION/test_localization_00816_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00817_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00818_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00819_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00820_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00821_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00822_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00823_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00824_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00825_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00826_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00827_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00828_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00829_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00830_prediction.png
SUBMISSION_LOCALIZATION/test_localization_00831_predict

In [9]:
import datetime
now = datetime.datetime.now()
this_time = now.strftime("%m_%d_%Y_%H_%M_%S")
submission_folder = "submission_{}".format(this_time)
os.mkdir(submission_folder)

In [10]:
import os
os.system("cp {}/* {}/".format(LOCALIZATION_SUBMISSION_FOLDER, submission_folder))
os.system("cp {}/* {}/".format(DAMAGE_SUBMISSION_FOLDER, submission_folder))
os.system("zip -r {}.zip {}".format(submission_folder, submission_folder))

0

In [11]:
print(submission_folder)

submission_12_22_2019_17_41_57
